In [1]:
!pip install PyDrive
!pip install rouge

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import json, gzip
import torch
import torch.nn as nn
import numpy as np
import time
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import re
import unicodedata

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Data for training set = 10k dev set = 1k
download = drive.CreateFile({'id': '1yJB36Jj1jw3U0JwE3IwhBYDY3VI1oCGd'})
download.GetContentFile('train.p')
download = drive.CreateFile({'id': '1T6YxqByvjNlCSZrU7FVmdCYlxM0RHlgw'})
download.GetContentFile('dev.p')
download = drive.CreateFile({'id': '1Nja8rXISqKfTSEbIPKYaf6Ey_Ud_Fik7'})
download.GetContentFile('test.p')
download = drive.CreateFile({'id': '1vPOGkgumTOVQ4N84GBo8AAPeAgwfyMep'})
download.GetContentFile('index2word.p')
download = drive.CreateFile({'id': '1JRYv-RQqhX6-fdYVUmyZmP9tBOtZjp_w'})
download.GetContentFile('word2index.p')

In [0]:
# # Data for training set = 50k dev set = 5k
# download = drive.CreateFile({'id': '1xq4JlZoWZ8_eO5F6VNBQvk2Mvub1Etpb'})
# download.GetContentFile('trainL.p')
# download = drive.CreateFile({'id': '1G3YAtOtrfv3PswNSHyGm0SIZy60n95zU'})
# download.GetContentFile('devL.p')
# download = drive.CreateFile({'id': '1T7rj8WokLSEEbc7-nqH48mjSnwBt4hEq'})
# download.GetContentFile('testL.p')
# download = drive.CreateFile({'id': '1bZXADxMyEn-yWtneA1sxDouGpuce1LRh'})
# download.GetContentFile('index2wordL.p')
# download = drive.CreateFile({'id': '1IMnW7Oyq4q9Ho5z87MbmVwlODjzEvY9h'})
# download.GetContentFile('word2indexL.p')


In [0]:
# # #Data with Extractive/Abstractive lable
# download = drive.CreateFile({'id': '1LOzhGMuW8F4ChPUsycBjEOwpLDQPeD0I'})
# download.GetContentFile('trainNew.p')
# download = drive.CreateFile({'id': '1yoaCyqZgZ5QszX0g4ABnNV-9p-D6DUlZ'})
# download.GetContentFile('devNew.p')
# download = drive.CreateFile({'id': '1XSaEgXAwPWTojc6ZjzIy8vR38jjiCUSW'})
# download.GetContentFile('testNew.p')
# download = drive.CreateFile({'id': '1U45R7EOqcJCUQAyHMMRmhFgWoxfzxyL8'})
# download.GetContentFile('index2wordNew.p')
# download = drive.CreateFile({'id': '1DGnwq99nJAq9i9TQ45HPjaO3aYWqDDEL'})
# download.GetContentFile('word2indexNew.p')

In [0]:
train_dict = pickle.load( open( "train.p", "rb" ) )
dev_dict = pickle.load( open( "dev.p", "rb" ) )
test_dict = pickle.load(open("test.p", "rb"))
index2word = pickle.load( open( "index2word.p", "rb" ) )
word2index = pickle.load( open( "word2index.p", "rb" ) )

In [0]:
#Extractive summary for 100 data points
# download = drive.CreateFile({'id': '1N6K91ySpywAB9pSKVCoJZKBn84S-2UFx'})
# download.GetContentFile('extract_train100.p')
# train_dict = pickle.load( open( "extract_train100.p", "rb" ) )

In [0]:
# # data_code = "extractive"
# # data_code = "abstractive"
# data_code = "mixed"
# filtered_train_dict = []
# filtered_dev_dict = []
# filtered_test_dict = []

# for ind in range(len(train_dict)):
#     data = train_dict[ind]
#     if data['density_bin'] == data_code:
#         filtered_train_dict.append(data)
# for ind in range(len(dev_dict)):
#     data = dev_dict[ind]
#     if data['density_bin'] == data_code:
#         filtered_dev_dict.append(data)
# for ind in range(len(test_dict)):
#     data = test_dict[ind]
#     if data['density_bin'] == data_code:
#         filtered_test_dict.append(data)

# print(len(filtered_train_dict))
# print(len(filtered_dev_dict))
# print(len(filtered_test_dict))

# train_dict = filtered_train_dict[:]
# dev_dict = filtered_dev_dict[:]
# test_dict = filtered_test_dict[:]

In [0]:
train_dict = train_dict[:]
dev_dict = dev_dict[:]
test_dict = test_dict[:]

In [11]:
train_dict_text = []
for i, data in enumerate(train_dict):
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
#         if i==789:
#             print(s, summ)
    t['summary'] = summ
    train_dict_text.append(t)
print(len(train_dict_text))

dev_dict_text = []
for data in dev_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    dev_dict_text.append(t)
print(len(dev_dict_text))

test_dict_text = []
for data in dev_dict:
    t = {}
    text_ind = data['text'][:]
    summ_ind = data['summary'][:]
    text = ''
    summ = ''
    for txt in text_ind:
        if text:
            text += ' '
        text += index2word[txt]
    t['text'] = text
    for s in summ_ind:
        if summ:
            summ += ' '
        summ += index2word[s]
    t['summary'] = summ
    test_dict_text.append(t)
print(len(test_dict_text))

10000
1000
1000


In [12]:
ind = 10
print(train_dict[ind]['summary'])
print(train_dict_text[ind]['summary'])
print(index2word[20439])

[785, 786, 83, 791, 683, 792, 793, 38, 794, 19, 783, 784, 126, 218]
linsey godfrey was struck while walking down the street in los angeles monday afternoon
the%20amount%20of%20time%20it%20takes%20to%20foreclose%20on%20a%20home%20is%20increasing%2c%20meaning%20that%20delinquent%20borrowers%20can%20stay%20in%20their%20houses%20for%20months%20after%20they%20stop%20making%20payments%2c%20according%20to%20freddie%20mac


In [13]:
import nltk
import nltk.data
from nltk import tokenize
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, sys
import pickle
import json, gzip
import time
import math
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import euclidean
nltk.download('punkt')

W0509 23:12:19.873581 140058628908928 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [15]:
embed = hub.Module(module_url)
tf.logging.set_verbosity(tf.logging.ERROR)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Instructions for updating:
Colocations handled automatically by placer.


W0509 23:12:21.061535 140058628908928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    embed = hub.Module(module_url)
    embedded_text = embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

In [17]:
ext_train_data = []
ext_dev_data = []
ext_test_data = []

for i, data in enumerate(train_dict_text):
    if i%1000==0:
        print(i)
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    try:
        sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
        num_sent = sentences_embeddings.shape[0]
        k = round(math.sqrt(num_sent))
        clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
        cluster_centers = clustering.cluster_centers_
        center_dist = defaultdict(list)
        for ns in range(num_sent):
            label = clustering.labels_[ns]
            dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
            center_dist[label].append((dist, ns))

        final = []
        for labels in range(k):
            try:
                center_dist[labels].sort(key = lambda x: x[0])
                final.append(center_dist[labels][0][1])
            except:
                continue
        final.sort()
        ex = []
        for i in final:
            ex.append(sentences_list[i])
        extractive = (' ').join(ex)
        dict_ = {}
        dict_['text'] = extractive[:]
        dict_['summary'] = data['summary'][:]
        ext_train_data.append(dict_)
    except:
        print('error')
end = time.time()
print(end-start)

for i, data in enumerate(dev_dict_text):
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive[:]
    dict_['summary'] = data['summary'][:]
    ext_dev_data.append(dict_)
end = time.time()
print(end-start)

for i, data in enumerate(test_dict_text):
    start = time.time()
    text = data['text'][:]
    sentences_list = tokenize.sent_tokenize(text)
    #print((sentences_list))
    
    sentences_embeddings = session.run(embedded_text, feed_dict={text_input: sentences_list})
    num_sent = sentences_embeddings.shape[0]
    k = round(math.sqrt(num_sent))
    clustering = KMeans(n_clusters=k).fit(sentences_embeddings)
    cluster_centers = clustering.cluster_centers_
    center_dist = defaultdict(list)
    for ns in range(num_sent):
        label = clustering.labels_[ns]
        dist = euclidean(sentences_embeddings[ns], cluster_centers[label])
        center_dist[label].append((dist, ns))
            
    final = []
    for labels in range(k):
        try:
            center_dist[labels].sort(key = lambda x: x[0])
            final.append(center_dist[labels][0][1])
        except:
            continue
    final.sort()
    ex = []
    for i in final:
        ex.append(sentences_list[i])
    extractive = (' ').join(ex)
    dict_ = {}
    dict_['text'] = extractive[:]
    dict_['summary'] = data['summary'][:]
    ext_test_data.append(dict_)
end = time.time()
print(end-start)

0
1000
2000
3000
4000
5000
6000
error
7000
8000
9000
0.02487468719482422
0.025245189666748047
0.025255680084228516


In [0]:
ext_train_data = sorted(ext_train_data, key = lambda i: len(i['text']), reverse = True)
# dev_dict = sorted(dev_dict, key = lambda i: len(i['text']), reverse = True)

In [19]:
ind = 100
# print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])
# print(ext_dev_data[ind]['text'])
# print(ext_dev_data[ind]['summary'])

hoverboards are here , a year earlier than back to the future promised


In [0]:
from nltk.tokenize import word_tokenize
ind2word = {0: 'PAD', 1:'BOS', 2:'EOS', 3:'UNK'}
word2ind = {'PAD':0, 'BOS':1, 'EOS':2, 'UNK':3}
count = 4
for data in ext_train_data:
    text = data['text'][:]
    summ = data['summary'][:]
    for words in word_tokenize(text):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1

    for words in word_tokenize(summ):
        if words not in word2ind:
            word2ind[words] = count
            ind2word[count] = words
            count += 1

In [21]:
ind = 78
# print(ext_train_data[ind]['text'])
print(ext_train_data[ind]['summary'])

insert trail text here


In [22]:
train_dict = []
dev_dict = []
test_dict = []

for i, data in enumerate(ext_train_data):
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    
    for words in word_tokenize(text):
        text_ind.append(word2ind[words])
    data_point['text_len'] = len(text_ind)
    if len(text_ind)<200:
        text_ind += [0]*(200-len(text_ind))
        
    for words in word_tokenize(summ):
        summ_ind.append(word2ind[words])
    summ_ind = [1]+summ_ind+[2]
    if len(summ_ind)<30:
        summ_ind += [0]*(30-len(summ_ind))
        
    if len(summ_ind)>30 or len(text_ind)>200:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%', i)
    else:
        data_point['text'] = text_ind[:]
        data_point['summary'] = summ_ind[:]
        #print((data_point['text_len']))
        train_dict.append(data_point)
    
for data in ext_dev_data:
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    for words in word_tokenize(text):
        if words in word2ind:
            text_ind.append(word2ind[words])
        else:
            text_ind.append(word2ind['UNK'])
    data_point['text_len'] = len(text_ind)
    if len(text_ind)<200:
        text_ind += [0]*(200-len(text_ind))
    for words in word_tokenize(summ):
        if words in word2ind:
            summ_ind.append(word2ind[words])
        else:
            summ_ind.append(word2ind['UNK'])
    summ_ind = [1]+summ_ind+[2]
    if len(summ_ind)<30:
        summ_ind += [0]*(30-len(summ_ind))
    if len(summ_ind)>30 or len(text_ind)>200:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%', i)
    else:
        data_point['text'] = text_ind[:]
        data_point['summary'] = summ_ind[:]
        #print((data_point['text_len']))
        dev_dict.append(data_point)
        
for data in ext_test_data:
    data_point = {}
    text = data['text'][:]
    summ = data['summary'][:]
    text_ind = []
    summ_ind = []
    for words in word_tokenize(text):
        if words in word2ind:
            text_ind.append(word2ind[words])
        else:
            text_ind.append(word2ind['UNK'])
    data_point['text_len'] = len(text_ind)
    if len(text_ind)<200:
        text_ind += [0]*(200-len(text_ind))

    for words in word_tokenize(summ):
        if words in word2ind:
            summ_ind.append(word2ind[words])
        else:
            summ_ind.append(word2ind['UNK'])
    summ_ind = [1]+summ_ind+[2]
    if len(summ_ind)<30:
        summ_ind += [0]*(30-len(summ_ind))
    if len(summ_ind)>30 or len(text_ind)>200:
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%', i)
    else:
        data_point['text'] = text_ind[:]
        data_point['summary'] = summ_ind[:]
        #print((data_point['text_len']))
        test_dict.append(data_point)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 441
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 507
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 667
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 1164
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 4676
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 4743
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 4839
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 5273
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 5358
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 5645
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 5773
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 5784
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 6440
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 7860
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 8281
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 8999
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 9294
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [23]:
data = {}
data['train'] = train_dict[:]
data['dev'] = dev_dict[:]
data['test'] = test_dict[:]
s = (data['train'][90]['summary'])[:]
print(s)
st = ''
for ss in s:
    st += ind2word[ss]
    st += ' '
print(st)
#print(ind2word[184],ind2word[6398],ind2word[125],ind2word[93],ind2word[249],ind2word[18],ind2word[4500],ind2word[14327],ind2word[2586],ind2word[1815],ind2word[68],ind2word[660],ind2word[14] )

[1, 15, 654, 87, 32, 3943, 466, 265, 3944, 77, 3945, 3932, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
BOS the country has been plagued by a wave of sectarian killings EOS PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 


In [0]:
class EncoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.emb_size = params['emb_size']
        self.vocab_size = params['vocab_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True, num_layers=self.num_layers)

    def forward(self, input, hidden, memory, seq_lengths):
        batch, seq_len = input.size()
        emb = self.embedding(input)
        packed_emb = pack_padded_sequence(emb, (seq_lengths).cpu().numpy(), batch_first = True)
        packed_output, (h, c) = self.lstm(packed_emb, (hidden, memory))
        output, _ = pad_packed_sequence(packed_output, batch_first = True)
        return output, h, c

    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
class biEncoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(biEncoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.emb_size = params['emb_size']
        self.vocab_size = params['vocab_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True, num_layers=self.num_layers, bidirectional = True)

    def forward(self, input, hidden, memory, seq_lengths):
        batch, seq_len = input.size()
        emb = self.embedding(input)
        output, (h, c) = self.lstm(emb, (hidden, memory))
        output1 = output[ :, :, :self.hidden_size]
        output2 = output[:, :, self.hidden_size:] #reverse
        output = output1+output2
        h = h.view(self.num_layers, 2, batch, self.hidden_size)
        c = c.view(self.num_layers, 2, batch, self.hidden_size)
        h = h[:,0,:,:]
        c = c[:,0,:,:]
        return output, h, c

    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
class DecoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.vocab_size = params['vocab_size']
        self.emb_size = params['emb_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True)
        self.w = nn.Linear(self.hidden_size, self.vocab_size)

    def forward(self, input, hidden, memory):
        batch_size, seq_len = input.size()
        emb = self.embedding(input)
        output, (h, c) = self.lstm(emb, (hidden, memory))
        logits = self.w(output)
        return logits, h, c

    def initHidden(self):
        h_init = torch.randn(self.num_layers, batch_size, self.d_hid).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.d_hid).cuda()
        return h_init, c_init

In [0]:
class AttnDecoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.vocab_size = params['vocab_size']
        self.emb_size = params['emb_size']
        self.num_layers = params['num_layers']
        self.batch_size = params['batch_size']
        #self.dropout_p = params['dropout_p']

        self.embedding = nn.Embedding(self.vocab_size, self.hidden_size)
        #self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        #self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first=True, num_layers = self.num_layers)
        self.w = nn.Linear(2*self.hidden_size, self.vocab_size)

    def forward(self, input, hidden, memory, encoder_outputs, pad_ind):
        batch_size, seq_len = input.size()
        embedded = self.embedding(input)
        #embedded = self.dropout(embedded)
        
        output,(h,c) = self.lstm(embedded, (hidden, memory))
        
        attn_values = torch.bmm(output,encoder_outputs.permute(0,2,1))
        dec_attn_values = torch.bmm(output,output.permute(0,2,1))
    
        for b in range(batch_size):
            attn_values[b,:,pad_ind[b]:] = -1e10
        
        mask = torch.ones_like(dec_attn_values).cuda()
        mask1 = -1e10*torch.ones_like(dec_attn_values).cuda()
        mask = torch.tril(mask, diagonal = -1)
        mask1 = torch.triu(mask1)
        mask = mask+mask1
        dec_attn_values = dec_attn_values*mask.float()
        
        total_attn_values = torch.cat((attn_values,dec_attn_values),2)
        total_outputs = torch.cat((encoder_outputs,output),1)
        
        sftmax = nn.Softmax(dim=2)
        total_attn_weights = sftmax(total_attn_values)
        weight_attn = torch.bmm(total_attn_weights, total_outputs)
        final_hidden = torch.cat((output,weight_attn),2)
        scores = self.w(final_hidden)
        return scores, h, c


    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
from rouge import Rouge

def calculate_rouge (hyps, refs):
    #print(hyps, refs)
    r = Rouge()
    scores = r.get_scores(hyps, refs, avg=True)
    return(scores)


In [0]:
def train_lm(dataset, params, net, fscore = False):
    
    print(dataset[90]['summary'])
    # since the first index corresponds to the PAD token, we just ignore it
    # when computing the loss
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    sftmax = nn.Softmax(dim=2)
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    num_examples = len(dataset)    
    batches = [(start, start + params['batch_size']) for start in range(0, num_examples, params['batch_size'])]
    
    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        #random.shuffle(batches)
        ref_summ = []
        gen_summ = []
        count = 0
        # for each batch, calculate loss and optimize model parameters            
        for b_idx, (start, end) in enumerate(batches):
            batch = dataset[start:end]
            #print(start, end)
            batch_text = np.zeros([len(batch), len(batch[0]['text'])])
            batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
            batch_pad_len = np.zeros(len(batch))
            i = 0
            for d in batch:
                batch_text[i] = d['text']
                batch_summary[i] = d['summary']
                batch_pad_len[i] = d['text_len']
                i += 1
            
            batch_text = torch.from_numpy(batch_text).long().cuda()
            batch_summary = torch.from_numpy(batch_summary).long().cuda()
            batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()

            pred = net(batch_text, batch_summary, batch_pad_len)
            preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
            targets = batch_summary[:, 1:].contiguous().view(-1)
            loss = criterion(preds, targets)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss
            
            if fscore:
                logits = sftmax(pred)
                words = torch.argmax(logits, dim=2)
                for sent in range(words.size()[0]):
                    summ = ''
                    gold = ''
                    for word in range(words.size()[1]):
                        if ind2word[words[sent][word].item()] == 'EOS':
                            break
                        if summ :
                            summ += ' '
                        summ += ind2word[words[sent][word].item()]

                    for word in range(1,len(batch_summary[sent])-1):
                        if ind2word[batch_summary[sent][word].item()] == 'EOS':
                            break
                        if gold:
                            gold += ' '
                        gold += ind2word[batch_summary[sent][word].item()]

                    if summ == '.'*len(summ):
                        count += 1
                    if (not summ) or (summ == '.'*len(summ)):
                        summ = '====='
                    
                    if gold and summ and gold!='.'*len(gold):
                        gen_summ.append(summ[:])
                        ref_summ.append(gold[:])
        print(start)   
        ind = 0
        scores = pred[ind,:,:].unsqueeze(0)
        logits = sftmax(scores)
        words = torch.argmax(logits, dim=2).squeeze(0)
        gold_summ_last = ''
        summ_last = ''
        for l in range(batch_summary[ind,1:].size()[0]):
            gold_summ_last += ind2word[batch_summary[ind,l].item()] + ' '
        for l in range(words.size()[0]):
            summ_last += ind2word[words[l].item()] + ' '
        print('########################Train######################')
        print(gold_summ_last)
        print(summ_last)
        
        #print(len(gen_summ), len(ref_summ))
        if fscore:
            rouge_score = calculate_rouge(gen_summ, ref_summ)
            print('rouge', rouge_score)
        
#         print('epoch: %d, loss: %0.2f, time: %0.2f sec' %\
#                 (epoch, ep_loss, time.time()-start_time))
        loss, perplex =  compute_loss(data['dev'], net, params['batch_size'], fscore = True)
        print('epoch: %d, loss: %0.2f, time: %0.2f sec, dev loss: %0.2f, dev perplexity: %0.2f' %\
              (epoch, ep_loss, time.time()-start_time, loss, perplex))
        print('####################################################')

In [0]:
def compute_loss(dataset, net, bsz=10, fscore = False):
    
    sftmax= nn.Softmax(dim=2)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
    batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    
    total_unmasked_tokens = 0. # count how many unpadded tokens there are
    nll = 0.
    gen_summ = []
    ref_summ = []
    count = 0
    for b_idx, (start, end) in enumerate(batches):
#         print(start, end)
        batch = dataset[start:end]
        
        batch_text = np.zeros([len(batch), len(batch[0]['text'])])
        batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
        batch_pad_len = np.zeros([len(batch)])
        i = 0
        for d in batch:
            batch_text[i] = d['text']
            batch_summary[i] = d['summary']
            batch_pad_len[i] = d['text_len']
            i += 1
        batch_text = torch.from_numpy(batch_text).long().cuda()
        batch_summary = torch.from_numpy(batch_summary).long().cuda()
        batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
            
        ut = torch.nonzero(batch_text).size(0)
        pred = net(batch_text, batch_summary, batch_pad_len)
        targets = batch_summary[:, 1:].contiguous().view(-1)
        preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
        loss = criterion(preds, targets)
        nll += loss.detach()
        total_unmasked_tokens += ut
        
        if fscore:
            #print('pred', pred.size())
            logits = sftmax(pred)
            words = torch.argmax(logits, dim=2)
            #print('words', words.size())
            for sent in range(words.size()[0]):
                summ = ''
                gold = ''
                for word in range(words.size()[1]):
                    if ind2word[words[sent][word].item()] == 'EOS':
                        break
                    if summ :
                        summ += ' '
                    summ += ind2word[words[sent][word].item()]

                for word in range(1,len(batch_summary[sent])-1):
                    if ind2word[batch_summary[sent][word].item()] == 'EOS':
                        break
                    if gold:
                        gold += ' '
                    gold += ind2word[batch_summary[sent][word].item()]

                if summ == '.'*len(summ):
                    count += 1
                if not summ or summ == '.'*len(summ):
                    summ = '====='
                
                if gold and summ and gold!='.'*len(gold):
                    gen_summ.append(summ[:])
                    ref_summ.append(gold[:])
        
    ind = 0
    scores = pred[ind,:,:].unsqueeze(0)
    logits = sftmax(scores)
    words = torch.argmax(logits, dim=2).squeeze(0)
    gold_summ = ''
    summ = ''
    for l in range(batch_summary[ind,1:].size()[0]):
        gold_summ += ind2word[batch_summary[ind,l].item()] + ' '
    for l in range(words.size()[0]):
        summ += ind2word[words[l].item()] + ' '
            
    
    print('#############Dev###########')
    if fscore:
        rouge_score = calculate_rouge(gen_summ, ref_summ)
        print('rouge', rouge_score)
    print(gold_summ)
    print(summ)
    
    perplexity = torch.exp(nll / total_unmasked_tokens).cpu()
    return nll, perplexity.data


In [0]:
# # function to evaluate LM perplexity on some input data, DO NOT MODIFY
# def test_lm(dataset, net, bsz=1, beam_search = False, max_len = 30):
    
#     sftmax = nn.Softmax(dim=1)
#     criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
#     num_examples = len(dataset)  
#     # we'll still use batches b/c we can't fit the whole
#     # validation set into GPU memory
#     batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
#     nll = 0
#     ans = torch.zeros([num_examples,max_len+1])
#     for b_idx, (start, end) in enumerate(batches):
            
#         batch = dataset[start:end]
#         batch_text = np.zeros([len(batch), len(batch[0]['text'])])
#         batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
#         batch_pad_len = np.zeros([len(batch)])
#         i = 0
        
#         for d in batch:
#             batch_text[i] = d['text'][:]
#             batch_summary[i] = d['summary'][:]
#             batch_pad_len[i] = d['text_len']
#             i += 1
            
#         batch_text = torch.from_numpy(batch_text).long().cuda()
#         batch_summary = torch.from_numpy(batch_summary).long().cuda()
#         batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        
#         batch_summary_gen = torch.ones([bsz, 1], dtype=torch.long).cuda()
        
#         for i in range(max_len):    
#             preds = net(batch_text, batch_summary_gen, batch_pad_len)[:,-1,:]
#             distribution = sftmax(preds)
#             ind = torch.argmax(distribution,1).unsqueeze(1)
#             batch_summary_gen = torch.cat((batch_summary_gen,ind),1) 
        
#         ans[start:end,:] = batch_summary_gen.clone()
    
#     gen_summ = []
#     gold_summ = []
    
#     for n in range(num_examples):
#         summ = ''
#         for i in range(1,max_len+1):
#             if summ:
#                 summ += ' '
#             if int(ans[n,i]) == 2:
#                 break
#             summ += ind2word[int(ans[n,i])]
#         gen_summ.append(summ)
#         #print(gen_summ[n])
        
#     for n in range(num_examples):
#         summ = ''
#         for i in range(1,max_len+1):
#             if summ:
#                 summ += ' '
#             if int(dataset[n]['summary'][i]) == 2:
#                 break
#             summ += ind2word[int(dataset[n]['summary'][i])]
#         gold_summ.append(summ)
#     print(len(gold_summ))
#     print(len(gen_summ))
#     rouge_score = calculate_rouge(gen_summ, gold_summ)
#     print('rouge', rouge_score)
#     for n in range(num_examples):
#         print(gold_summ[n])
#         print(gen_summ[n])
#         print("----------------")

In [0]:
# function to evaluate LM perplexity on some input data, DO NOT MODIFY
def test_lm(dataset, net, bsz=1):
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
#     ipdb.set_trace()

    batches = [(start, start + bsz) for start in\
               range(0, num_examples, bsz)]
    
    nll = 0
    max_len = 60
    for b_idx, (start, end) in enumerate(batches):
            
        batch = dataset[start:end]
        batch_text = np.zeros([len(batch), len(batch[0]['text'])])
        batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
        batch_pad_len = np.zeros([len(batch)])
        i = 0
        ans = ''
        count = 0
        for d in batch:
            batch_text[i] = d['text']
            batch_summary[i] = d['summary']
            batch_pad_len[i] = d['text_len']
            i += 1
        batch_text = torch.from_numpy(batch_text).long().cuda()
        batch_summary = torch.from_numpy(batch_summary).long().cuda()
        batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        batch_summary_test = torch.ones([bsz, 1], dtype=torch.long).cuda()
        for i in range(max_len):    
            preds = net(batch_text, batch_summary_test, batch_pad_len)
            preds = preds.contiguous().view(-1, net.vocab_size)
            m = nn.Softmax(dim=1)
            distribution = m(preds)
#             print(distribution.size())
            tx = torch.argmax(distribution,1)
#             print(tx)
            temp = tx[-1]
#             print(temp.item())
#             temp = torch.multinomial(distribution,1)[-1]
#             print(temp.size())
            temp = temp.cpu()
            x = torch.ones([bsz, 1], dtype=torch.long).cuda()
            x[0][0] = temp.item()
            if(ind2word[temp.item()] == "EOS"):
                break
            ans += ind2word[temp.item()] + ' '
            count += 1
#             print(ind2word[temp.item()])
            if(ind2word[temp.item()] == "EOS"):
                break
            batch_summary_test = torch.cat((batch_summary_test,x),1) 
        print("Insise test_lm: ")
        print(ans)
#         targets = batch_summary[:, 1:].contiguous().view(-1)
#         preds = preds[:-1, :].contiguous().view(-1, net.vocab_size)
#         loss = criterion(preds, targets)
#         nll += loss.detach()

    return ans, count

In [0]:
# # function to evaluate LM perplexity on some input data, DO NOT MODIFY
# def test_lm_bms(dataset, net, bsz=1):
    
#     net1 = net
#     net2 = net
    
#     criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
#     num_examples = len(dataset)  
    
#     batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    
#     nll = 0
#     max_len = 22
#     for b_idx, (start, end) in enumerate(batches):
            
#         batch = dataset[start:end]
#         batch_text = np.zeros([len(batch), len(batch[0]['text'])])
#         batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
#         batch_pad_len = np.zeros([len(batch)])
#         i = 0
#         ans = []
#         for d in batch:
#             batch_text[i] = d['text']
#             batch_summary[i] = d['summary']
#             batch_pad_len[i] = d['text_len']
#             i += 1
#         batch_text = torch.from_numpy(batch_text).long().cuda()
#         gold_batch_summary = torch.from_numpy(batch_summary).long().cuda()
#         batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
        
#         dec_batch_summary1 = torch.ones([bsz, 1], dtype=torch.long).cuda()
#         dec_batch_summary2 = torch.ones([bsz, 1], dtype=torch.long).cuda()
#         for i in range(max_len):    
            
#             if dec_batch_summary1[0][-1]==2 and dec_batch_summary2[0][-1]==2:
#                 break
#             else:
#                 if dec_batch_summary1[0][-1]!=2:
#                     preds1 = net1(batch_text, dec_batch_summary1, batch_pad_len)
#                     preds1 = preds1.contiguous().view(-1, net.vocab_size)
#                     sftmax = nn.Softmax(dim=1)
#                     distribution1 = sftmax(preds1)
#                     prob1, top_words1 = torch.topk(distribution1[-1],k=2,dim=0)
#                     #print('1',prob1, top_words1)

#                 else:
#                     prob1 = -1e40*torch.ones_like(prob1)

#                 if dec_batch_summary2[0][-1]!=2:
#                     preds2 = net2(batch_text, dec_batch_summary2, batch_pad_len)
#                     preds2 = preds2.contiguous().view(-1, net.vocab_size)
#                     sftmax = nn.Softmax(dim=1)
#                     distribution2 = sftmax(preds2)
#                     prob2, top_words2 = torch.topk(distribution2[-1],k=2,dim=0)
#                     #print('2',prob2, top_words2)

#                 else:
#                     prob2 = -1e40*torch.ones_like(prob2)

#                 prob_cat = torch.cat((prob1, prob2))
#                 top_words_cat = torch.cat((top_words1, top_words2))
#                 final_top_words = torch.topk(prob_cat,k=2,dim=0)[1]
#                 #print(prob_cat, final_top_words)
#                 #print(dec_batch_summary1)

#                 if final_top_words[0] in [0,1]:
#                     dec_batch_summary1 = torch.cat((dec_batch_summary1,top_words1[final_top_words[0]].view(-1,1)),1) 
#                 elif final_top_words[0] in [2,3]:
#                     dec_batch_summary1 = torch.cat((dec_batch_summary2,top_words2[final_top_words[0]-2].view(-1,1)),1)
#                 if final_top_words[1] in [0,1]:
#                     dec_batch_summary2 = torch.cat((dec_batch_summary1,top_words1[final_top_words[1]].view(-1,1)),1) 
#                 elif final_top_words[0] in [2,3]:
#                     dec_batch_summary2 = torch.cat((dec_batch_summary2,top_words2[final_top_words[1]-2].view(-1,1)),1)
            
#             #print(dec_batch_summary1)
#             #print(ind2word[temp.item()])
#             #if(ind2word[temp.item()] == "EOS"):
#             #    break
#            # batch_summary_test = torch.cat((batch_summary_test,x),1) 
# #         targets = batch_summary[:, 1:].contiguous().view(-1)
# #         preds = preds[:-1, :].contiguous().view(-1, net.vocab_size)
# #         loss = criterion(preds, targets)
# #         nll += loss.detach()
    
#         summ_str1 = ''
#         summ_str2 = ''
#         for l in range(dec_batch_summary1.size()[1]):
#             summ_str1 += ind2word[dec_batch_summary1[0][l].item()]+' '
#         for l in range(dec_batch_summary2.size()[1]):
#             summ_str2 += ind2word[dec_batch_summary2[0][l].item()]+' '
    
#     return nll, summ_str1,summ_str2

In [0]:
class SuperAwesome(nn.Module):
    def __init__(self, params):
        super(SuperAwesome, self).__init__()
        #self.enc = EncoderLSTM(params).cuda()
        self.bienc = biEncoderLSTM(params).cuda()
        #self.dc = DecoderLSTM(params).cuda()
        self.attn_dc = AttnDecoderLSTM(params).cuda()
        self.vocab_size = params['vocab_size']
    def forward(self, batch_text, batch_summary, batch_pad_len):
        batch_size, seq_size = batch_text.size()
        #print(batch_size)
#         h_init,c_init = self.enc.initHidden(batch_size)
#         enc_output,h_enc,c_enc = self.enc(batch_text,h_init,c_init, batch_pad_len)
        
        h_init,c_init = self.bienc.initHidden(batch_size)
        enc_output,h_enc,c_enc = self.bienc(batch_text,h_init,c_init, batch_pad_len)
        #h_init,c_init = self.attn_dc.initHidden(batch_size)
        h_init, c_init = h_enc.contiguous(),c_enc.contiguous()
        attn_pre,h,c = self.attn_dc(batch_summary,h_init,c_init,enc_output, batch_pad_len)
#         attn_pre, h, c = self.dc(batch_summary, h_enc, c_enc)
        return attn_pre

In [46]:
params = {}
params['hidden_size'] =512
params['emb_size'] =512
params['num_layers'] = 1
params['batch_size'] = 40
params['vocab_size'] = len(word2ind)
params['learning_rate'] = 0.001
params['epochs'] = 8
net = SuperAwesome(params)

train_lm(data['train'], params, net, fscore = True)

[1, 15, 654, 87, 32, 3943, 466, 265, 3944, 77, 3945, 3932, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
9960
########################Train######################
BOS big cities aren 't the only places people drive high-end vehicles . EOS PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 
the , , , the best is is is is the the EOS EOS . in in in in in in in in in in in in in in in 
rouge {'rouge-1': {'f': 0.14763291163907166, 'p': 0.2980992626834322, 'r': 0.10676027737690773}, 'rouge-2': {'f': 0.008945782097298656, 'p': 0.011433326908216348, 'r': 0.007745547121489297}, 'rouge-l': {'f': 0.10632934747850252, 'p': 0.27750858276659895, 'r': 0.09875446043806134}}
#############Dev###########
rouge {'rouge-1': {'f': 0.14536171548513088, 'p': 0.22155053557553528, 'r': 0.11467752545853122}, 'rouge-2': {'f': 0.00859346737457846, 'p': 0.009906820385496852, 'r': 0.007852254625319634}, 'rouge-l': {'f': 0.11710499451549469, 'p': 0.21016449661449632, 'r': 0.10909798412073173}}
BOS mo

In [47]:
test_len = []
#TEST TIME
gen_summ = []
ref_summ = []
for i in range(len(data['test'])):
    y = data['test'][i]['summary']
    gold = ''
    for t in y:
        if(ind2word[t]=='BOS'):
            continue
        if(ind2word[t] == 'EOS'):
            break  
        gold += (ind2word[t])
        gold +=' '
    
  #print()  
    print("ground truth")  
    print(gold)
    net.eval()
    summ, l = test_lm(data['test'][i:i+1], net) 
    test_len.append(l)
    if summ == '.'*len(summ):
        count += 1
    if not summ or summ == '.'*len(summ):
        summ = '====='

    gen_summ.append(summ)
    ref_summ.append(gold)
rouge_score = calculate_rouge(gen_summ, ref_summ)
print('Test rouge', rouge_score)  
# for n in range(num_examples):
#         summ = ''
#         for i in range(1,max_len+1):
#             if summ:
#                 summ += ' '
#             if int(ans[n,i]) == 2:
#                 break
#             summ += ind2word[int(ans[n,i])]
#         gen_summ.append(summ)

ground truth
the star 's aunt was held at gunpoint early wednesday at his tarzana home , police confirm to people 
Insise test_lm: 
it 's a pony and kylie 
ground truth
zillow ceo spencer UNK is optimistic when it comes to facebook and twitter . 
Insise test_lm: 
you 're thinking of credit card . 
ground truth
uber is setting its sights on further economic expansion after raising $ 1 .6 billion in convertible debt . 
Insise test_lm: 
the singer dished on newlywed 's relationship with a `` workout '' 
ground truth
this fuller house horror movie trailer isn 't exactly what you 'd expect from `` your old familiar friends '' 
Insise test_lm: 
the singer dished on newlywed 's relationship with a `` workout '' 
ground truth
margulies memorably played new york mayor lenny UNK in ghostbusters 
Insise test_lm: 
`` it 's a joke from the last week 's fabulous 
ground truth
when UNK group was founded in 1991 , video conferencing was an UNK business product . 
Insise test_lm: 
a joke from the comed